# Reporting Discovery - Deep Dive

> This builds on the early discovery work by attempting to consolidate both the ***definition*** and ***instance*** trees to generate meaningful displayable data.
>


### 🎗️ Reminder, these are the two main structures of our data hierarchy:
>
> ### Run-time(Instances)
> =18 -> 10 -> 19
>       +11 -> 09
>             +15
>             +08 -> 12
>                   +13
>                   +14
> ### Structural (Definitions)
> =07 -> 20 -> 01
>       +21
>       +06 -> 17
>       +08
>       +02
>       +05 -> 04 -> 16
> 
> In the previous study we used a library to simplify extraction of two-level > nested data, and developed an outline template of a recursive function that > could in theory build arbitrarily deep nested queries. We do not at this stage > know how meaningful very deep queries might be, but it is most likely that > edited versions of these would be very valuable in revealing information from > both the design-time and run-time data when filtered appropriately.
> 
> Because the algorithm has been successful, it is feasible to simply flesh out > the main recursive function to build the sql using the same reflective > techniques as where used in the two-layer nesting.
> 
> This will allow us to generate the queries needed to drive the main design-time > and run-time queries and decide how best to combine and separate these for best > usability.
> 
> Once the processor is working it should be incorporated in the raw library.

### Load the main library.

In [1]:
// |> load libraries ...
#i "file:///n:/nix-nuget/"
#r "nuget:nix-library-raw" 
#r "nuget:RazorInteractive" 

Restore sources file:///n:/nix-nuget/ Installed Packages nix-library-raw, 1.1.4 RazorInteractive, 1.2.0

Loading extensions from `C:\Users\NickMajor\.nuget\packages\razorinteractive\1.2.0\lib\net8.0\RazorInteractive.dll`

Renders the code block as Razor markup in dotnet-interactive notebooks. 
 This extension adds a new kernel that can render Razor markdown. 
 All C# and F# variables are available in the @Model property.

### Connect the database

Using the library, 

- read the connection string from the environment
- create the JsonSqlClient
- as a test of connectivity, Get the column names from the rpt.reports table
- Display the result.

In [2]:
// |> housekeeping needed to set up the library
using Nix.Library.Raw;
string connectionString = System.Environment.GetEnvironmentVariable("Nix_DB_DG");
JsonSqlClient sql = new JsonSqlClient(connectionString);

// verify that the libraries and the sql connection work
var x = sql.GetColumnNames("rpt.reports");
x.Display();

[ [ReportId], [OrganizationId], [ReportType], [FileNameTemplate], [Enabled], [Format], [TextQualifier], [TransmitterConfiguration], [Parameters], [ReportMethodId], [ReportTitle], [DateModified], [ReportInstanceId], [IsScheduledInApp] ]

## Definitions

To represent the two main divergent data paths through the metadata, we define an instanceData variable and a definitionData variable.

> The method used to generate the sql in the prior discovery used two fully qualified column names to get the rest of its information using refection.
>
> The method used here regards each related pair of column names as a single `Link`, and allows us to build arbitrary lengths of such links.
>
> This allows us to define an instanceData object and a definitionData object containing the two branches of the data hierarchy.

In [3]:
// |> define the links for both trees

// for future work, these are the two main data sources

/*
=18 -> 10 -> 19
      +11 -> 09
            +15
            +08 -> 12
                  +13
                  +14
*/
// The data chain for investigating specific instances:
var instanceData = new Link[] { // these are PRIMARY <- FOREIGN
    new Link("rpt.ReportTimeRanges.ReportTimeRangeId", "rpt.ReportInstances.TimePeriod", 2 ), //18
    new Link("rpt.Reports.ReportId", "rpt.ReportTransmitterConfigLinks.ReportId", 2 ), //10
    new Link("rpt.ReportTransmitterConfigs.ReportTransmitterConfigId", "rpt.ReportTransmitterConfigLinks.ReportTransmitterConfigId", 2), //19
    new Link("rpt.ReportInstances.ReportInstanceId", "rpt.Reports.ReportInstanceId", 2 ), //11
    new Link("rpt.Reports.ReportId", "rpt.ReportSchedule.ReportId", 2  ), //9
    new Link("rpt.ReportMethods.Id", "rpt.Reports.ReportMethodId", 2 ), //15
    new Link("rpt.Reports.ReportId", "rpt.ReportLogs.ReportId", 2 ), //8
    new Link("rpt.ReportLogs.ReportLogId", "rpt.ReportActivityLog.ReportLogId", 2 ), //12
    new Link("rpt.ReportLogs.ReportLogId", "rpt.ReportFileLog.ReportLogId", 2 ), //13
    new Link("rpt.ReportLogs.ReportLogId", "rpt.ReportTransmissionLogs.ReportLogId", 2 ), //14
};
/* //! todo: fix the main diagram to match whatever ends up wprking here.
=07 -> 20 -> 01
      +21
      +06 -> 17
      +08
      +02
      +05 -> 04 -> 16
      +00
      +03
      */ 
// the data chain for investigating definitions:
var definitionData = new Link[] {
    new Link ("rpt.ReportDefinitions.ReportDefinitionId", "rpt.ReportInstances.ReportDefinitionId", 2 ), //7
    new Link ("rpt.ReportTypes.ReportTypeId", "rpt.ReportDefinitions.ReportTypeId", 2 ), //20
    new Link ("rpt.ReportCategories.ReportCategoryId", "rpt.ReportTypes.ReportCategoryId", 2 ), //1
    new Link ("rpt.ReportDefinitions.ReportDefinitionId", "rpt.ReportDefinitionCustomAttributes.ReportDefinitionId", 2 ), //21
    new Link ("rpt.ReportDefinitions.ReportDefinitionId", "rpt.ReportDefinitionColumns.ReportDefinitionId", 2 ), //6
    new Link ("rpt.ReportSearchableTermTypes.ReportSearchableTermTypeId", "rpt.ReportDefinitionColumns.ReportSearchableTermTypeId", 2 ), //17
    new Link ("rpt.Reports.ReportId", "rpt.ReportLogs.ReportId", 2 ), //8
    new Link ("rpt.ReportColumns.ReportColumnId", "rpt.ReportDefinitionColumns.ReportColumnId", 2 ), //2
    new Link ("rpt.ReportDataTypes.ReportDataTypeId", "rpt.ReportColumns.ReportDataTypeId", 2 ), //3
    new Link ("rpt.ReportDataTypes.ReportDataTypeId", "rpt.ReportDefinitionColumns.ReportDataTypeIdOverride", 2 ), //5
    new Link ("rpt.ReportAggregateTypes.ReportAggregateTypeId", "rpt.ReportDefinitionColumns.ReportAggregateTypeId", 2 ), //0
    new Link ("rpt.ReportDataTypes.ReportDataTypeId", "rpt.ReportDataTypeOperators.ReportDataTypeId", 5 ), //4
    new Link ("rpt.ReportOperatorTypes.ReportoperatorTypeId", "rpt.ReportDataTypeOperators.ReportOperatorTypeId", 2 ), //16
};


### Publishing for the Discovery of Instances and Definitions 

Having defined the links, we can use the GenerateNestedQuery method to make sql, which we can then apply to generate Json data, which we can then pass to a razor page for local display.

> For convenience we can save the sql and the json for later off-line use
> 
> The sql generator also generates a diagnostic output that describes the way the link sequence was interpretted.  This can be useful for diagnosing weird behavior from specifying the wrong start node, for example.
> 
> We display that here as a snity check.

In [4]:
// |> Generate the sql and query to retrieve the results as json

using Nix.Library.Raw;

List<string> Pages = new List<string>();

Nest.ToFile(Nest.GetCss(), "../.data/deep-dive.css");
// Definitions: generate and save sql
var definitionSql = sql.GenerateNestedQuery(definitionData);
definitionSql[1].Display();
Nest.ToFile(definitionSql[0], "../.data/definitions-raw.sql");
// generate and save json
var definitionJson = sql.JsonSingleUsingQuery(definitionSql[0]);
Nest.ToFile(definitionJson, "../.data/definitions-raw.json");
// insstances: cregenerate and save sql
var instanceSql = sql.GenerateNestedQuery(instanceData);
instanceSql[1].Display();
Nest.ToFile(instanceSql[0], "../.data/instances-raw.sql");
// generate and svae json
var instanceJson = sql.JsonSingleUsingQuery(instanceSql[0]);
Nest.ToFile(instanceJson, "../.data/instances-raw.json");



Root: -- ReportDefinitions <+--++ ReportInstances --
Peer: -- ReportTypes <+--++ ReportDefinitions --
Nest: -- ReportTypes <+--++ ReportCategories --
Peer: -- ReportDefinitionCustomAttributes <+--++ ReportDefinitions --
Peer: -- ReportDefinitionColumns <+--++ ReportDefinitions --
Nest: -- ReportDefinitionColumns <+--++ ReportSearchableTermTypes --
Peer: -- ReportColumns <+--++ ReportDefinitionColumns --
Nest: -- ReportColumns <+--++ ReportDataTypes --
Nest: -- ReportDataTypes <+--++ ReportDefinitionColumns --
Nest: -- ReportDefinitionColumns <+--++ ReportAggregateTypes --
Peer: -- ReportDataTypeOperators <+--++ ReportDataTypes --
Nest: -- ReportDataTypeOperators <+--++ ReportOperatorTypes --

Root: -- ReportTimeRanges <+--++ ReportInstances --
Nest: -- ReportInstances <+--++ Reports --
Nest: -- Reports <+--++ ReportTransmitterConfigLinks --
Nest: -- ReportTransmitterConfigLinks <+--++ ReportTransmitterConfigs --
Peer: -- ReportSchedule <+--++ Reports --
Peer: -- ReportMethods <+--++ Reports --
Peer: -- ReportLogs <+--++ Reports --
Nest: -- ReportLogs <+--++ ReportActivityLog --
Peer: -- ReportFileLog <+--++ ReportLogs --
Peer: -- ReportTransmissionLogs <+--++ ReportLogs --

In [5]:
//|> render all to html files and display in browser
// rendering these from file allows us to repeat this step on its own while refining the presentation settings which we likewise save to a file
Nest.NestOptions options = new Nest.NestOptions
{
    DarkMode = false,
    MaximumCellPixelWidth = 10000,
    MaximumPixelWidth = 10000,
    TitleColumnPixelWidth = 120
};
var y = Nest.GetCss(options);
//x.Display();
Nest.ToFile(y, "../.html/deep-dive.css");
Pages.Add(Nest.RenderHtmlStepping(Nest.FromFile("../.data/definitions-raw.json"), 10, "Sample Report Definitions"));
Pages.Add(Nest.RenderHtmlStepping(Nest.FromFile("../.data/instances-raw.json"), 10, "Sample Report Instances"));

Nest.ToFile(Nest.RenderPageStepping(definitionJson, 5, "Report Definitions", options), "./Definitions.html");
Nest.ToFile(Nest.RenderPageStepping(instanceJson, 5, "Report Instances", options), "./Instances.html");
"Written to file".Display();

Nest.RunFile(".\\Definitions.html");
Nest.RunFile(".\\Instances.html");
"Launched in browser".Display();

Written to file

Launched in browser

### Quick view using Razor

The pages can be also viewed in a Razor page for a quick check. 

⚠️ This has some limitations, for example the stepping version does not execute live here so we use a scrolling version of the page (not yet fully investigated but possibly needs the javascript kernel initialized ina cell).

ℹ️ Html can also be pasted into an html cell though, and runs in the notebook.

In [6]:
// |> save nests to file as html - Definitions and Instances.
Nest.ToFile(Nest.RenderPageScrolling(definitionJson, "DEFS", 5, "Report Definitions", options), "./Definitions.html");
Nest.ToFile(Nest.RenderPageScrolling(instanceJson, "DEFS", 5, "Report Instances", options), "./Instances.html");

// |> save nests to file as html - Definitions and Instances.
Nest.ToFile(Nest.RenderPageScrolling(definitionJson, "DEFS", 5, "Report Definitions", options), "./Definitions.html");
Nest.ToFile(Nest.RenderPageScrolling(instanceJson, "DEFS", 5, "Report Instances", options), "./Instances.html");

In [7]:
<!-- publish in razor page -->
#!razor

<html>
  <head>
    <link rel="stylesheet" href="./deep-dive.css">
  </head>
  <body>
        @Raw(@Model.Pages[0])
        <hr>
        @Raw(@Model.Pages[1])
      <hr>
  </body>
</html>

<!-- publish in razor page -->

Sample Report Definitions ReportDefinitions 
 1/2 2/2 ReportDefinitionId 33 32 ReportTypeId 33 32 CreatedDate 2024-04-03T23:09:13.9923440 2024-01-29T17:11:12.2229288 ModifiedDate ReportInstance ReportInstance 
 1/2 2/2 ReportInstanceId 6852 6883 ReportDefinitionId 33 33 ReportTypeId 33 33 OrganizationId 1 1 IsSavedPreset false false PresetTitle ApprovalsDetailedByUser_Title ReportingInApp_ReportType_ApprovalsDetailedByUser_Title TimePeriod 9 9 CustomStartDate CustomEndDate FilteredData {"ReportDefinitionId":33,"VisibleColumns":[{"Id":"main_516","ReportDefinitionColumnId":516,"UseAggregate":false,"Order":1},{"Id":"main_514","ReportDefinitionColumnId":514,"UseAggregate":false,"Order":2},{"Id":"main_531","ReportDefinitionColumnId":531,"UseAggregate":false,"Order":3},{"Id":"main_533","ReportDefinitionColumnId":533,"UseAggregate":false,"Order":4},{"Id":"main_526","ReportDefinitionColumnId":526,"UseAggregate":false,"Order":5},{"Id":"main_517","ReportDefinitionColumnId":517,"UseAggregate":false,"Order":6},{"Id":"main_528","ReportDefinitionColumnId":528,"UseAggregate":false,"Order":7},{"Id":"main_535","ReportDefinitionColumnId":535,"UseAggregate":false,"Order":8},{"Id":"main_513","ReportDefinitionColumnId":513,"UseAggregate":false,"Order":9},{"Id":"main_509","ReportDefinitionColumnId":509,"UseAggregate":false,"Order":10},{"Id":"main_521","ReportDefinitionColumnId":521,"UseAggregate":false,"Order":11},{"Id":"main_527","ReportDefinitionColumnId":527,"UseAggregate":false,"Order":12},{"Id":"main_536","ReportDefinitionColumnId":536,"UseAggregate":false,"Order":13},{"Id":"main_507","ReportDefinitionColumnId":507,"UseAggregate":false,"Order":14},{"Id":"main_508","ReportDefinitionColumnId":508,"UseAggregate":false,"Order":15},{"Id":"main_511","ReportDefinitionColumnId":511,"UseAggregate":false,"Order":16},{"Id":"main_512","ReportDefinitionColumnId":512,"UseAggregate":false,"Order":17},{"Id":"main_529","ReportDefinitionColumnId":529,"UseAggregate":false,"Order":18},{"Id":"main_506","ReportDefinitionColumnId":506,"UseAggregate":false,"Order":19},{"Id":"main_530","ReportDefinitionColumnId":530,"UseAggregate":false,"Order":20},{"Id":"main_519","ReportDefinitionColumnId":519,"UseAggregate":false,"Order":21},{"Id":"main_532","ReportDefinitionColumnId":532,"UseAggregate":false,"Order":22},{"Id":"main_510","ReportDefinitionColumnId":510,"UseAggregate":false,"Order":23},{"Id":"main_522","ReportDefinitionColumnId":522,"UseAggregate":false,"Order":24},{"Id":"main_520","ReportDefinitionColumnId":520,"UseAggregate":false,"Order":25},{"Id":"main_525","ReportDefinitionColumnId":525,"UseAggregate":false,"Order":26},{"Id":"main_515","ReportDefinitionColumnId":515,"UseAggregate":false,"Order":27},{"Id":"main_524","ReportDefinitionColumnId":524,"UseAggregate":false,"Order":28},{"Id":"main_505","ReportDefinitionColumnId":505,"UseAggregate":false,"Order":29},{"Id":"main_518","ReportDefinitionColumnId":518,"UseAggregate":false,"Order":30},{"Id":"main_534","ReportDefinitionColumnId":534,"UseAggregate":false,"Order":31},{"Id":"main_523","ReportDefinitionColumnId":523,"UseAggregate":false,"Order":32}],"TimeRange":{"Id":"main_516","ReportDefinitionColumnId":516,"IsCustom":false,"ReportTimeRangeId":9,"StartDateTime":null,"EndDateTime":null},"FilterColumns":[{"Id":"main_516","ReportDefinitionColumnId":516,"DataTypeOperatorId":0,"IsForTimeRange":true}],"ConditionGroups":[],"CustomAttributeColumns":[],"SegmentIds":[]} {"ReportDefinitionId":33,"VisibleColumns":[{"Id":"main_516","ReportDefinitionColumnId":516,"UseAggregate":false,"Order":1},{"Id":"main_514","ReportDefinitionColumnId":514,"UseAggregate":false,"Order":2},{"Id":"main_531","ReportDefinitionColumnId":531,"UseAggregate":false,"Order":3},{"Id":"main_533","ReportDefinitionColumnId":533,"UseAggregate":false,"Order":4},{"Id":"main_526","ReportDefinitionColumnId":526,"UseAggregate":false,"Order":5},{"Id":"main_517","ReportDefinitionColumnId":517,"UseAggregate":false,"Order":6},{"Id":"main_528","ReportDefinitionColum